<a href="https://colab.research.google.com/github/smatiolids/vector-dbs/blob/main/YT_video_transcription_for_generating_knowledge_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Transcription to fill knowledge base

I talked to a customer who wants to leverage influencers comments to recommend their products.

The idea then is to process public youtube videos, collecting influencer's speech to use their voice to generate responses and indicate the video content which was the source of the respoonse.

The Hybrid Search would be important to find specific products, categories or product's types that are related to the company business.

This is part of a demo I will do to a customer, so I tried to solve both the learning and the demo in one shot.

### The video

I choose a video from a Brazilian influencer answering questions about science. It has a lot of different subjects, from science, astronomy and curiosities.

### References:

https://code.pieces.app/blog/how-to-download-a-youtube-video-in-mp3-format-with-python
https://lablab.ai/t/whisper-tutorial



### Before start

- Set runtime type as GPU

## Installing Whisper and FFMPEG

In [1]:
# Install whisper.ai
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tq8mu0l9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tq8mu0l9
  Resolved https://github.com/openai/whisper.git to commit 0a60fcaa9b86748389a656aa013c416030287d47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230918-py3-none-any.whl size=798399 sha256=277b53f08923edc79833c84ee52d061b7801a76d8453eb7fa30efc63fcd6b5a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-gzalpj2d/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [39]:
!sudo apt update && sudo apt install ffmpeg

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E

In [3]:
import torch
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

# Importing Youtube audio


In [75]:
!pip install pytube moviepy mock pydub python-dotenv cassio openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00


In [5]:
import os
import pytube
from moviepy.editor import *
from pydub import AudioSegment

In [27]:
def split_mp3(input_file, output_directory, segment_duration_sec, overlap_sec = 1):
    # Load the MP3 file
    audio = AudioSegment.from_mp3(input_file)
    segment_duration_ms = segment_duration_sec * 1000
    overlap_ms = overlap_sec * 1000
    # Calculate the number of segments
    num_segments = len(audio) // ( segment_duration_ms - overlap_ms)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Split the MP3 file into segments
    for i in range(num_segments):
        start_time = i * ( segment_duration_ms - overlap_ms)
        if start_time < 0:
          start_time = 0

        end_time = (i + 1) * segment_duration_ms
        segment = audio[start_time:end_time]

        # Define the output filename
        output_file = os.path.join(output_directory, f"segment_{str(i + 1).zfill(5)}.mp3")

        # Export the segment as an MP3 file
        segment.export(output_file, format="mp3")

        print(f"Segment {i + 1} saved as {output_file}")

    return True



In [7]:
# Solve pytube issue: https://github.com/pytube/pytube/issues/1498
import re
import mock

from pytube.cipher import get_throttling_function_code

def patched_throttling_plan(js: str):
    """Patch throttling plan, from https://github.com/pytube/pytube/issues/1498"""
    raw_code = get_throttling_function_code(js)

    transform_start = r"try{"
    plan_regex = re.compile(transform_start)
    match = plan_regex.search(raw_code)

    #transform_plan_raw = find_object_from_startpoint(raw_code, match.span()[1] - 1)
    transform_plan_raw = js

    # Steps are either c[x](c[y]) or c[x](c[y],c[z])
    step_start = r"c\[(\d+)\]\(c\[(\d+)\](,c(\[(\d+)\]))?\)"
    step_regex = re.compile(step_start)
    matches = step_regex.findall(transform_plan_raw)
    transform_steps = []
    for match in matches:
        if match[4] != '':
            transform_steps.append((match[0],match[1],match[4]))
        else:
            transform_steps.append((match[0],match[1]))

    return transform_steps

def download_youtube_audio(video_url, output_path):
  try:
    with mock.patch('pytube.cipher.get_throttling_plan', patched_throttling_plan):
      import pytube
      # Create a YouTube object
      yt = pytube.YouTube(video_url)

      # Get the highest resolution stream
      video_stream = yt.streams.filter(only_audio=True).first()

      # Download the audio stream
      video_stream.download(output_path=output_path)

      # Convert the downloaded file to MP3
      mp4_file_path = os.path.join(output_path, video_stream.default_filename)
      mp3_file_path = os.path.splitext(mp4_file_path)[0] + ".mp3"

      video_clip = AudioFileClip(mp4_file_path)
      video_clip.write_audiofile(mp3_file_path)

      # Delete the original MP4 file
      os.remove(mp4_file_path)

      print(f"Audio downloaded and saved as {mp3_file_path}")
      return True
  except Exception as e:
    print(f"An error occurred: {str(e)}")
    return False

## Downloading a video from youtube

In [66]:
download_youtube_audio("https://www.youtube.com/watch?v=KDwnyem8tk4",".")

MoviePy - Writing audio in ./Bandeira na Lua dá para ver da Terra Iberê responde!.mp3


MoviePy - Done.
Audio downloaded and saved as ./Bandeira na Lua dá para ver da Terra Iberê responde!.mp3


True

## Splitting audio file in chunks of 29 seconds, with an overlap of 1 second

In [78]:
video = "Bandeira na Lua dá para ver da Terra Iberê responde!.mp3"

In [71]:
split_mp3(video,"audio",29)

Segment 1 saved as audio/segment_00001.mp3
Segment 2 saved as audio/segment_00002.mp3
Segment 3 saved as audio/segment_00003.mp3
Segment 4 saved as audio/segment_00004.mp3
Segment 5 saved as audio/segment_00005.mp3
Segment 6 saved as audio/segment_00006.mp3
Segment 7 saved as audio/segment_00007.mp3
Segment 8 saved as audio/segment_00008.mp3
Segment 9 saved as audio/segment_00009.mp3
Segment 10 saved as audio/segment_00010.mp3
Segment 11 saved as audio/segment_00011.mp3
Segment 12 saved as audio/segment_00012.mp3
Segment 13 saved as audio/segment_00013.mp3
Segment 14 saved as audio/segment_00014.mp3
Segment 15 saved as audio/segment_00015.mp3
Segment 16 saved as audio/segment_00016.mp3
Segment 17 saved as audio/segment_00017.mp3
Segment 18 saved as audio/segment_00018.mp3
Segment 19 saved as audio/segment_00019.mp3
Segment 20 saved as audio/segment_00020.mp3
Segment 21 saved as audio/segment_00021.mp3
Segment 22 saved as audio/segment_00022.mp3
Segment 23 saved as audio/segment_00023.m

True

## Transcribing Audio


In [ ]:
# Running whisper from command line
#!whisper "Comentando comentários  Drauzio Comenta 00.mp3" --language Portuguese --model medium

In [31]:
import whisper
import numpy as np
import pandas as pd

model = whisper.load_model("medium", device=DEVICE)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 762,321,920 parameters.


In [70]:
#!rm -r ./audio
#!rm ./Comentando*

rm: cannot remove './audio': No such file or directory


In [72]:
def decode_audio(input_directory):
  import whisper
  import numpy as np

  model = whisper.load_model("medium", device=DEVICE)
  res = []
  for filename in sorted(os.listdir(input_directory)):
    if filename.endswith(".mp3"):
      print(filename)
      audio = whisper.load_audio(f"""{input_directory}/{filename}""" )
      audio = whisper.pad_or_trim(audio)
      mel = whisper.log_mel_spectrogram(audio).to(model.device)
      options = whisper.DecodingOptions(language="pt")
      result = whisper.decode(model, mel, options)
      res.append({
          'segment': filename,
          'text':result.text
      })
  return res


In [73]:
# print the recognized text
decoded = decode_audio("./audio")

segment_00001.mp3
segment_00002.mp3
segment_00003.mp3
segment_00004.mp3
segment_00005.mp3
segment_00006.mp3
segment_00007.mp3
segment_00008.mp3
segment_00009.mp3
segment_00010.mp3
segment_00011.mp3
segment_00012.mp3
segment_00013.mp3
segment_00014.mp3
segment_00015.mp3
segment_00016.mp3
segment_00017.mp3
segment_00018.mp3
segment_00019.mp3
segment_00020.mp3
segment_00021.mp3
segment_00022.mp3
segment_00023.mp3
segment_00024.mp3
segment_00025.mp3
segment_00026.mp3
segment_00027.mp3
segment_00028.mp3
segment_00029.mp3
segment_00030.mp3
segment_00031.mp3
segment_00032.mp3
segment_00033.mp3
segment_00034.mp3
segment_00035.mp3
segment_00036.mp3
segment_00037.mp3
segment_00038.mp3
segment_00039.mp3
segment_00040.mp3
segment_00041.mp3
segment_00042.mp3
segment_00043.mp3
segment_00044.mp3
segment_00045.mp3
segment_00046.mp3
segment_00047.mp3
segment_00048.mp3
segment_00049.mp3
segment_00050.mp3
segment_00051.mp3
segment_00052.mp3
segment_00053.mp3


In [81]:
df = pd.DataFrame(decoded)
df.head()

,segment,text
0,segment_00001.mp3,"Iberê, é possível ver aqui da terra, usando um..."
1,segment_00002.mp3,superpoderes e ele também não era um cavalo. E...
2,segment_00003.mp3,bícorns? Vaca? Alce? É que que que como que a ...
3,segment_00004.mp3,38 mil anos atrás já existiam seres humanos aí...
4,segment_00005.mp3,"é o Narval, ele é uma espécie de uma baleia, q..."


# Enter Astra

I would use CassIO, but as it doesn;t have the support to analyzers (yet), I will use regular CQL

In [82]:
import os
from dotenv import find_dotenv, load_dotenv
dotenv_file = find_dotenv('.env')
load_dotenv(dotenv_file, override=True)

True

In [49]:
#import cassio
#cassio.init(token=os.environ["ASTRA_DB_APPLICATION_TOKEN"], database_id=os.environ["ASTRA_DB_ID"])

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(133345985551744) b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.db.astra.datastax.com:29042:e92b9c1c-6c60-4f91-8742-1c43b0145d22> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [62]:
# Astra Connection (Based on cqlsession, from cassio library)
import os
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

def getCQLSession(mode='astra_db'):
    print('Initializing CQL Session')
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"],
            },
            auth_provider=PlainTextAuthProvider(
                os.environ["ASTRA_DB_CLIENT_ID"],
                os.environ["ASTRA_DB_CLIENT_SECRET"],
            ),
        )
        astraSession = cluster.connect(os.environ["ASTRA_DB_KEYSPACE"])
        astraSession.row_factory = dict_factory
        print('Connected')
        return astraSession
    elif mode == 'local':
        cluster = Cluster()
        localSession = cluster.connect()
        return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    return os.environ["ASTRA_DB_KEYSPACE"]


In [63]:
# Open the cennection with Astra
session = getCQLSession()

Initializing CQL Session


ERROR:cassandra.connection:Closing connection <AsyncoreConnection(133345537501904) b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.db.astra.datastax.com:29042:05d703a1-936c-48cc-9ed8-c195f51b1c07> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Connected


In [64]:
keyspace = os.environ["ASTRA_DB_KEYSPACE"]
table = f"""yt_audio_texts"""
from cassandra.cqlengine.query import BatchStatement
from cassandra.query import BatchType


cmd_create_table = f"""CREATE TABLE IF NOT EXISTS {table} (
    id text,
    decoded text,
    embedding vector<float, 1536>,
    PRIMARY KEY (id)
)"""

cmd_create_ix_vector = f"""CREATE CUSTOM INDEX IF NOT EXISTS {table}_emb ON {table} (embedding)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{'similarity_function': 'DOT_PRODUCT' }}"""

cmd_create_ix_analyzer = f"""CREATE CUSTOM INDEX IF NOT EXISTS {table}_an ON {table} (decoded)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'WITH OPTIONS = {{ 'index_analyzer': 'standard', 'case_sensitive': false}}"""

In [65]:
session.execute(cmd_create_table)
session.execute(cmd_create_ix_vector)
session.execute(cmd_create_ix_analyzer)

In [77]:
cmd_insert = f"""
INSERT INTO {table}  (id, decoded, embedding)
VALUES (:id, :decoded, :embedding)
"""
prepared_stmt = session.prepare(cmd_insert)

In [89]:
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']

In [94]:
# Generating embeddings
for index, row in df.iterrows():
  session.execute(prepared_stmt,{'id': f"""{video}_{index}""", 'decoded': row["text"], 'embedding': generate_embedding(row["text"],)})

# Searching mentions to the planet Saturn

In the case we want to find mentions to an specific term, like a product, the analyzer works better then the embeddings.

Lets search for mentions to the planet Saturn

In [116]:
# Query with parameter
cmd_select_1 = f"""
select id, decoded from {table}
where decoded : :query
limit 5;
"""
prepared_select_1 = session.prepare(cmd_select_1)
rs = session.execute(prepared_select_1,{"query": "saturno"})
list(rs)

[{'id': 'Bandeira na Lua dá para ver da Terra Iberê responde!.mp3_31',
  'decoded': 'diferentes em volta de Saturno. Então, se tem meio que umas rodovias gigantes em volta de Saturno, girando em velocidades diferentes, é uma coisa bem maluca. A gente não sabe exatamente como eles se formaram, mas as hipóteses aí são de que uma lua se chocou com a outra, acabou ficando em órbita em volta de Saturno, ou um cometa, um asteroide que formou aquela poeira que a gente vê ali, que de longe é muito bonito. Quem viu primeiro esses anéis foi Galileu em mil seiscentos e dez. E é muito legal porque ele não'},
 {'id': 'Bandeira na Lua dá para ver da Terra Iberê responde!.mp3_32',
  'decoded': 'ver muito bem parecia um planeta com orelhas e ele mesmo usou essa palavra orelhas pra descrever o que ele tava vendo ali. E outra coisa interessante é que não é só Saturno que tem anéis, tá? Júpiter, Urano e Netuno também tem anéis, mas eles não são tão visíveis, assim, eles não são tão elegantes quanto os an

# Running with similarity

Now, lets ask for contents related to Saturn

In [117]:
# Query with parameter
query = "Saturno"

cmd_select_2 = f"""
select id, decoded from {table}
ORDER BY embedding ANN OF :embedding
limit 5;
"""
prepared_select_2 = session.prepare(cmd_select_2)
rs = session.execute(prepared_select_2,{"embedding": generate_embedding(query)})
list(rs)

[{'id': 'Bandeira na Lua dá para ver da Terra Iberê responde!.mp3_32',
  'decoded': 'ver muito bem parecia um planeta com orelhas e ele mesmo usou essa palavra orelhas pra descrever o que ele tava vendo ali. E outra coisa interessante é que não é só Saturno que tem anéis, tá? Júpiter, Urano e Netuno também tem anéis, mas eles não são tão visíveis, assim, eles não são tão elegantes quanto os anéis de Saturno. Felipe, que tá atrás da câmera gravando, acabou de fazer uma pergunta muito boa. Ele perguntou se o lixo espacial não poderia formar um anel no planeta Terra. Até poderia, mas a questão é que o lixo não'},
 {'id': 'Bandeira na Lua dá para ver da Terra Iberê responde!.mp3_31',
  'decoded': 'diferentes em volta de Saturno. Então, se tem meio que umas rodovias gigantes em volta de Saturno, girando em velocidades diferentes, é uma coisa bem maluca. A gente não sabe exatamente como eles se formaram, mas as hipóteses aí são de que uma lua se chocou com a outra, acabou ficando em órbita e

In this case, more information related to Saturn was retrieved

# Combining both

Ln this case, I have only three records mentioning Saturn, but if I have to choose only one record we can combine both


In [119]:
# Query with parameter
query = "Saturno"

cmd_select_3 = f"""
select id, decoded from {table}
where decoded : :query
ORDER BY embedding ANN OF :embedding
limit 1;
"""
prepared_select_3 = session.prepare(cmd_select_3)
rs = session.execute(prepared_select_3,{"query":query , "embedding": generate_embedding(query)})
list(rs)

[{'id': 'Bandeira na Lua dá para ver da Terra Iberê responde!.mp3_32',
  'decoded': 'ver muito bem parecia um planeta com orelhas e ele mesmo usou essa palavra orelhas pra descrever o que ele tava vendo ali. E outra coisa interessante é que não é só Saturno que tem anéis, tá? Júpiter, Urano e Netuno também tem anéis, mas eles não são tão visíveis, assim, eles não são tão elegantes quanto os anéis de Saturno. Felipe, que tá atrás da câmera gravando, acabou de fazer uma pergunta muito boa. Ele perguntou se o lixo espacial não poderia formar um anel no planeta Terra. Até poderia, mas a questão é que o lixo não'}]